# Recherche du meilleur modèle
Charge successivement les poids des modèles fine tuned et teste ces valeurs sur le jeu de messages annotés
Le modèle avec le meilleur score f1 est gardé.

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import tensorflow as tf
from sklearn import metrics
import seaborn as sns
from transformers import TFCamembertForSequenceClassification
import transformers.models.camembert.tokenization_camembert as tk

In [15]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 905839099126918692,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2249614951
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 15951191490776824107
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"]

In [16]:
model_name = "jplu/tf-camembert-base"

In [17]:
def encode_reviews(tokenizer, reviews, max_length):
    token_ids = np.zeros(shape=(len(reviews), max_length),
                         dtype=np.int32)
    for i, review in enumerate(reviews):
        encoded = tokenizer.encode(review, max_length=max_length)
        token_ids[i, 0:len(encoded)] = encoded
    attention_mask = (token_ids != 0).astype(np.int32)
    return {"input_ids": token_ids, "attention_mask": attention_mask}

In [18]:
def prediction(x, mdl):
    scores = mdl.predict(x)
    y_pred = np.argmax(scores['logits'], axis=1)
    return y_pred

In [19]:
def evaluation(y,y_pred,graph=True):
    TP = np.count_nonzero(y*y_pred)
    TN = np.count_nonzero((y-1)*(y_pred-1))
    FN = np.count_nonzero((y-1)*y_pred)
    FP = np.count_nonzero(y*(y_pred-1))
    tot = TP+TN+FN+FP

    accuracy = metrics.accuracy_score(y, y_pred)
    f1_score = metrics.f1_score(y, y_pred)
    conf_mx = metrics.confusion_matrix(y, y_pred)

    conf_matrix = [[TN,FN],[FP,TP]]
    if((TP+FP) != 0 and (TP+FN) != 0):
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1 = 2 * precision * recall / (precision + recall)
    else:
        precision = accuracy
        recall = -1
        f1 = f1_score

    print(f"Precision: {round(100 * precision,2)}")
    print(f"Rappel: {round(100 * recall,2)}")
    print(f"F-score: {round(100 * f1,2)}")
    print(f"%bon: {round(100*(TP+TN)/tot,2)} - %faux: {round(100*(FP+FN)/tot,2)}")
    print(f"Matrice: {conf_matrix}")
    if(graph):
        sns.heatmap(conf_matrix)
    return [precision,recall,f1,conf_matrix]

In [20]:
tokenizer = tk.CamembertTokenizer.from_pretrained(model_name,do_lower_case=True)
assert tokenizer != None

In [21]:
data_messages = pd.read_csv("./messages/messages0.csv")
msg_labels = data_messages['sentiments'].values.tolist()
msg = data_messages['messages'].values.tolist()

encoded_msg = encode_reviews(tokenizer, msg, 200)
y_msg = np.array(msg_labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [22]:
model1 = TFCamembertForSequenceClassification.from_pretrained(model_name)
olds = ["f169_count8000_epo5_batch4.h5","f171_count5000_epo8_batch24.h5","f172_count500_epo8_batch24.h5","f177_count3680_epo3_batch12.h5","f176_count3680_epo10_batch14.h5","f179_count8000_epo7_batch4.h5","f179_count1000_epo8_batch24.h5","f180_count1000_epo5_batch8.h5","f180_count10000_epo15_batch8.h5","f180_count36800_epo10_batch16.h5","f181_count720_epo20_batch4.h5","f193_count8000_epo7_batch4_allo.h5","f194_count8000_epo10_batch4_allo.h5"]
models_weights = [
    "f145_count2500_epo15_batch24.h5", 
    "f153_count1000_epo15_batch16.h5",
    "f179_count1440_epo20_batch24.h5",
    "f179_count8000_epo18_batch4.h5",
    "f179_count8000_epo20_batch4.h5",
    "f179_count12000_epo10_batch8.h5",
    "f182_count720_epo20_batch24.h5",
    "f184_count522_epo10_batch16.h5",
    "f184_count720_epo20_batch16.h5",
    "f191_count8000_epo3_batch4_allo.h5"]
models_score = []

All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

Some layers of TFCamembertForSequenceClassification were not initialized from the model checkpoint at jplu/tf-camembert-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
best = [0,0,0,0]
best_name = ""
for w in models_weights:
    model1.load_weights(f"./models_weights/{w}")
    print(f'\n--- {w}')
    y_pred = prediction(encoded_msg, model1)
    score = evaluation(y_msg,y_pred,graph=False)
    if(score[2]<0.77): print("-")
    elif(score[2]<0.775): print("")
    elif(score[2]<0.78): print("+")
    elif(score[2]<0.785): print("+++")
    elif(score[2]<0.79): print("+++++")
    elif(score[2]<0.80): print("*")
    else: print("***")

    if(score[2] > best[2]):
        best = score
        best_name = w
    models_score.append(score)


--- f145_count2500_epo15_batch24.h5
Precision: 81.07
Rappel: 75.69
F-score: 78.29
%bon: 72.66 - %faux: 27.34
Matrice: [[65, 44], [32, 137]]
+++

--- f153_count1000_epo15_batch16.h5
Precision: 86.39
Rappel: 71.57
F-score: 78.28
%bon: 70.86 - %faux: 29.14
Matrice: [[51, 58], [23, 146]]
+++

--- f179_count1440_epo20_batch24.h5
Precision: 82.84
Rappel: 72.92
F-score: 77.56
%bon: 70.86 - %faux: 29.14
Matrice: [[57, 52], [29, 140]]
+

--- f179_count8000_epo18_batch4.h5
Precision: 84.02
Rappel: 74.74
F-score: 79.11
%bon: 73.02 - %faux: 26.98
Matrice: [[61, 48], [27, 142]]
*

--- f179_count8000_epo20_batch4.h5
Precision: 84.02
Rappel: 75.13
F-score: 79.33
%bon: 73.38 - %faux: 26.62
Matrice: [[62, 47], [27, 142]]
*

--- f179_count12000_epo10_batch8.h5
Precision: 79.88
Rappel: 75.42
F-score: 77.59
%bon: 71.94 - %faux: 28.06
Matrice: [[65, 44], [34, 135]]
+

--- f182_count720_epo20_batch24.h5
Precision: 82.84
Rappel: 73.3
F-score: 77.78
%bon: 71.22 - %faux: 28.78
Matrice: [[58, 51], [29, 140]]
+

In [24]:
print(best_name)
print(best)
# f179_count8000_epo18_batch4.h5
# [0.8373493975903614, 0.7473118279569892, 0.7897727272727273, [[61, 47], [27, 139]]]

f179_count8000_epo20_batch4.h5
[0.8402366863905325, 0.7513227513227513, 0.7932960893854748, [[62, 47], [27, 142]]]
